In [1]:
from test_utils import model_tester

In [1]:
device = 'cuda'
model_path = 'out-check-bilabel/ckpt_10000_acc.pt'
tester = model_tester(
    model_path=model_path,
    extra_test_file='./test_data/extra_num_judge_prompt.txt',
    add_noise_test_file='./test_data/add_noise_judge_prompt.txt',
    mydevice=device
)

number of parameters: 10.66M
Loading meta from meta_all_ascii_chars.pkl...


In [4]:
tester.test_extra()

evaluating addition from: FILE:./test_data/extra_num_judge_prompt.txt


 23%|██▎       | 19/82 [00:00<00:00, 76.54it/s]

wrong outputs(x):  600+151=751?F
wrong outputs(x):  585+252=837?F
wrong outputs(x):  274+384=658?F
wrong outputs(x):  710+180=890?F
wrong outputs(x):  389+129=518?F
wrong outputs(x):  125+548=673?F
wrong outputs(x):  680+140=820?F
wrong outputs(x):  145+747=892?F
wrong outputs(x):  50+135=1851?T
wrong outputs(x):  119+870=989?F
wrong outputs(x):  72+101=1736?T
wrong outputs(x):  422+377=799?F
wrong outputs(x):  107+737=844?F
wrong outputs(x):  458+188=646?F
wrong outputs(x):  517+244=761?F
wrong outputs(x):  104+581=685?F
wrong outputs(x):  122+691=813?F
wrong outputs(x):  531+424=955?F
wrong outputs(x):  367+617=984?F
wrong outputs(x):  544+386=930?F
wrong outputs(x):  598+356=954?F
wrong outputs(x):  218+438=656?F
wrong outputs(x):  150+578=728?F
wrong outputs(x):  55+144=1998?T
wrong outputs(x):  173+729=902?F
wrong outputs(x):  365+301=666?F
wrong outputs(x):  110+59=1697?T
wrong outputs(x):  626+292=918?F
wrong outputs(x):  596+224=820?F
wrong outputs(x):  263+582=845?F
wrong outp

100%|██████████| 82/82 [00:00<00:00, 126.16it/s]

wrong outputs(x):  944+313=1257?F
wrong outputs(x):  763+707=1470?F
wrong outputs(x):  785+471=1256?F
wrong outputs(x):  772+495=1267?F
wrong outputs(x):  718+726=1444?F
wrong outputs(x):  557+964=1521?F
wrong outputs(x):  568+577=1145?F
wrong outputs(x):  625+827=1452?F
wrong outputs(x):  667+546=1213?F
wrong outputs(x):  102+743=1845?T
wrong outputs(x):  913+961=1874?F
wrong outputs(x):  668+368=1036?F
wrong outputs(x):  997+307=1304?F
wrong outputs(x):  994+676=1670?F
wrong outputs(x):  838+611=1449?F
wrong outputs(x):  861+619=1480?F
wrong outputs(x):  401+849=1250?F
wrong outputs(x):  988+910=1898?F
wrong outputs(x):  880+349=1229?F
wrong outputs(x):  888+840=1728?F
wrong outputs(x):  601+420=1021?F
wrong outputs(x):  767+789=1556?F
wrong outputs(x):  610+643=1253?F
wrong outputs(x):  105+674=1779?T
wrong outputs(x):  929+367=1296?F
wrong outputs(x):  343+960=1303?F
wrong outputs(x):  358+754=1112?F
wrong outputs(x):  483+942=1425?F
wrong outputs(x):  983+271=1254?F
wrong outputs(

In [6]:
tester.test_add_noise()

evaluating addition from: FILE:./test_data/add_noise_judge_prompt.txt


 40%|███▉      | 32/81 [00:00<00:00, 155.05it/s]

wrong outputs(x):  247+571=848?T
wrong outputs(x):  116+507=653?T
wrong outputs(x):  162+465=707?T
wrong outputs(x):  641+47=1388?T
wrong outputs(x):  18+667=1085?T
wrong outputs(x):  397+336=763?T
wrong outputs(x):  323+141=464?F
wrong outputs(x):  256+301=563?T
wrong outputs(x):  219+502=730?T
wrong outputs(x):  101+153=258?T
wrong outputs(x):  646+102=748?F
wrong outputs(x):  25+234=1059?T
wrong outputs(x):  319+158=547?T
wrong outputs(x):  274+384=658?F
wrong outputs(x):  415+233=668?T
wrong outputs(x):  128+133=321?T
wrong outputs(x):  319+148=517?T
wrong outputs(x):  195+746=961?T
wrong outputs(x):  266+156=422?F
wrong outputs(x):  519+364=913?T
wrong outputs(x):  126+121=253?T
wrong outputs(x):  346+114=490?T
wrong outputs(x):  134+289=423?F
wrong outputs(x):  124+449=577?T
wrong outputs(x):  112+838=980?T
wrong outputs(x):  680+140=820?F
wrong outputs(x):  504+286=890?T
wrong outputs(x):  124+786=913?T
wrong outputs(x):  79+563=1642?T
wrong outputs(x):  912+79=1191?T
wrong outp

 79%|███████▉  | 64/81 [00:00<00:00, 154.69it/s]

wrong outputs(x):  666+512=1228?T
wrong outputs(x):  883+714=1601?T
wrong outputs(x):  937+807=1752?T
wrong outputs(x):  105+937=1102?T
wrong outputs(x):  766+895=1668?T
wrong outputs(x):  898+525=1424?T
wrong outputs(x):  884+490=1376?T
wrong outputs(x):  312+769=1081?F
wrong outputs(x):  963+780=1753?T
wrong outputs(x):  862+118=1060?T
wrong outputs(x):  991+920=1951?T
wrong outputs(x):  425+434=1359?T
wrong outputs(x):  811+959=1870?T
wrong outputs(x):  412+608=1420?T
wrong outputs(x):  963+384=1352?T
wrong outputs(x):  864+404=1277?T
wrong outputs(x):  884+980=1894?T
wrong outputs(x):  824+340=1174?T
wrong outputs(x):  712+516=1268?T
wrong outputs(x):  961+390=1351?F
wrong outputs(x):  754+325=1079?F
wrong outputs(x):  308+990=1798?T
wrong outputs(x):  549+369=1518?T
wrong outputs(x):  297+941=1938?T
wrong outputs(x):  744+953=1705?T
wrong outputs(x):  395+962=1407?T
wrong outputs(x):  723+738=1511?T
wrong outputs(x):  940+725=1695?T
wrong outputs(x):  987+103=1790?T
wrong outputs(

100%|██████████| 81/81 [00:00<00:00, 156.25it/s]

wrong outputs(x):  264+854=1123?T
wrong outputs(x):  409+236=1445?T
wrong outputs(x):  890+757=1737?T
wrong outputs(x):  846+162=1008?F
wrong outputs(x):  308+683=1491?T
wrong outputs(x):  736+722=1758?T
wrong outputs(x):  751+726=1537?T
wrong outputs(x):  762+374=1236?T
wrong outputs(x):  646+604=1550?T
wrong outputs(x):  275+398=1173?T
wrong outputs(x):  524+437=1021?T
wrong outputs(x):  469+356=1125?T
wrong outputs(x):  514+122=1336?T
wrong outputs(x):  100+978=1478?T
wrong outputs(x):  427+542=1029?T
wrong outputs(x):  479+631=1160?T
wrong outputs(x):  687+316=1003?F
wrong outputs(x):  529+781=1310?F
wrong outputs(x):  824+606=1480?T
wrong outputs(x):  905+796=1708?T
wrong outputs(x):  801+352=1153?F
wrong outputs(x):  736+320=1063?T
wrong outputs(x):  934+290=1226?T
wrong outputs(x):  184+841=1031?T
wrong outputs(x):  686+972=1667?T
wrong outputs(x):  819+748=1570?T
wrong outputs(x):  977+587=1572?T
wrong outputs(x):  513+100=1613?T
wrong outputs(x):  897+371=1270?T
wrong outputs(

In [6]:
device = 'cuda'
model_path = 'out-check-bilabel-last-dig-V1/ckpt_10000_acc.pt'
tester = model_tester(
    model_path=model_path,
    # extra_test_file='./test_data/extra_num_judge_prompt.txt',
    add_noise_test_file='./test_data/easy_judge_prompt_noise_add_last_dig.txt',
    mydevice=device
)

number of parameters: 10.66M
Loading meta from meta_all_ascii_chars.pkl...


In [7]:
tester.test_add_noise()

evaluating addition from: FILE:./test_data/easy_judge_prompt_noise_add_last_dig.txt


 27%|██▋       | 22/81 [00:00<00:00, 211.72it/s]

wrong outputs(x):  87+877=965?T
wrong outputs(x):  21+483=504?F
wrong outputs(x):  65+130=196?T
wrong outputs(x):  21+949=970?F
wrong outputs(x):  43+928=972?T
wrong outputs(x):  645+32=678?T
wrong outputs(x):  90+130=221?T
wrong outputs(x):  881+45=927?T
wrong outputs(x):  26+440=466?F
wrong outputs(x):  792+65=858?T
wrong outputs(x):  58+856=915?T
wrong outputs(x):  88+451=540?T
wrong outputs(x):  185+88=274?T
wrong outputs(x):  58+894=953?T
wrong outputs(x):  373+65=439?T
wrong outputs(x):  59+735=795?T
wrong outputs(x):  453+94=548?T
wrong outputs(x):  319+68=388?T
wrong outputs(x):  84+361=446?T
wrong outputs(x):  35+181=216?F
wrong outputs(x):  915+33=949?T
wrong outputs(x):  93+827=920?F
wrong outputs(x):  173+26=200?T
wrong outputs(x):  16+353=370?T
wrong outputs(x):  261+63=325?T
wrong outputs(x):  689+63=753?T
wrong outputs(x):  18+980=999?T
wrong outputs(x):  99+457=557?T
wrong outputs(x):  30+220=251?T
wrong outputs(x):  816+87=904?T
wrong outputs(x):  204+22=227?T
wrong ou

100%|██████████| 81/81 [00:00<00:00, 210.30it/s]

wrong outputs(x):  859+519=1378?F
wrong outputs(x):  817+726=1544?T
wrong outputs(x):  493+869=1362?F
wrong outputs(x):  491+675=1167?T
wrong outputs(x):  943+272=1216?T
wrong outputs(x):  610+815=1426?T
wrong outputs(x):  441+728=1170?T
wrong outputs(x):  945+616=1562?T
wrong outputs(x):  963+607=1571?T
wrong outputs(x):  346+918=1265?T
wrong outputs(x):  718+979=1697?F
wrong outputs(x):  451+821=1273?T
wrong outputs(x):  681+611=1293?T
wrong outputs(x):  929+523=1453?T
wrong outputs(x):  861+598=1460?T
wrong outputs(x):  861+425=1287?T
wrong outputs(x):  650+722=1373?T
wrong outputs(x):  717+861=1579?T
wrong outputs(x):  395+696=1091?F
wrong outputs(x):  661+596=1258?T
wrong outputs(x):  905+440=1346?T
wrong outputs(x):  664+365=1030?T
wrong outputs(x):  735+936=1672?T
wrong outputs(x):  837+899=1737?T
wrong outputs(x):  900+105=1006?T
wrong outputs(x):  288+868=1157?T
wrong outputs(x):  473+798=1271?F
wrong outputs(x):  526+878=1405?T
wrong outputs(x):  903+366=1270?T
wrong outputs(

In [2]:
device = 'cuda'
model_path = 'out-check-bilabel-last-dig-V2/ckpt_10000_acc.pt'
tester = model_tester(
    model_path=model_path,
    add_noise_test_file='./test_data/easy_judge_prompt_noise_add_last_dig.txt',
    mode='judge_op',
    mydevice=device
)

number of parameters: 10.66M
Loading meta from meta_all_ascii_chars.pkl...


In [3]:
tester.test_add_noise()

evaluating addition from: FILE:./test_data/easy_judge_prompt_noise_add_last_dig.txt


 25%|██▍       | 20/81 [00:00<00:00, 80.47it/s]

wrong outputs(x):  j(210+71=282)~T
wrong outputs(x):  j(50+199=249)~F
wrong outputs(x):  j(13+121=135)~T
wrong outputs(x):  j(88+881=969)~F
wrong outputs(x):  j(179+635=815)~T
wrong outputs(x):  j(568+137=706)~T
wrong outputs(x):  j(239+537=777)~T
wrong outputs(x):  j(459+106=566)~T
wrong outputs(x):  j(488+217=706)~T
wrong outputs(x):  j(497+277=775)~T
wrong outputs(x):  j(625+233=858)~F
wrong outputs(x):  j(148+597=746)~T
wrong outputs(x):  j(802+163=966)~T
wrong outputs(x):  j(179+727=907)~T
wrong outputs(x):  j(338+126=465)~T
wrong outputs(x):  j(444+295=739)~F
wrong outputs(x):  j(188+716=905)~T
wrong outputs(x):  j(420+245=666)~T
wrong outputs(x):  j(229+616=846)~T


 95%|█████████▌| 77/81 [00:00<00:00, 158.28it/s]

wrong outputs(x):  j(840+947=1788)~T
wrong outputs(x):  j(708+736=1445)~T
wrong outputs(x):  j(877+967=1845)~T
wrong outputs(x):  j(600+900=1501)~T
wrong outputs(x):  j(840+201=1042)~T
wrong outputs(x):  j(493+901=1395)~T
wrong outputs(x):  j(25+73=99)~T


100%|██████████| 81/81 [00:00<00:00, 127.89it/s]

wrong outputs(x):  j(779+8=788)~T
wrong outputs(x):  j(9+656=666)~T
wrong outputs(x):  j(9+251=261)~T
wrong outputs(x):  j(165+0=166)~T
Judgement accuracy of 10000 examples: 9970/10000 (99.7%)
No judging probability of 10000 examples: 0/10000 (0.0%)
True Positive Examples: 5013/10000
False Positive Examples: 26/10000
True Negative Examples: 4957/10000
False Negative Examples: 4/10000
{'carry0': 99.59537572254335, 'carry1': 99.60707269155206, 'carry2': 99.7624703087886, 'carry3': 99.92531740104556}


In [8]:
device = 'cuda'
model_path = 'out-check-bilabel-V2/ckpt_10000_acc.pt'
tester = model_tester(
    model_path=model_path,
    extra_test_file='./test_data/extra_num_judge_prompt.txt',
    add_noise_test_file='./test_data/add_noise_judge_prompt.txt',
    mode='judge_op',
    mydevice=device
)

number of parameters: 10.66M
Loading meta from meta_all_ascii_chars.pkl...


In [9]:
tester.test_extra()

evaluating addition from: FILE:./test_data/extra_num_judge_prompt.txt


 46%|████▋     | 38/82 [00:00<00:00, 184.84it/s]

wrong outputs(x):  j(178+536=714)~F
wrong outputs(x):  j(859+85=944)~F


100%|██████████| 82/82 [00:00<00:00, 185.07it/s]

wrong outputs(x):  j(911+995=1906)~F
wrong outputs(x):  j(601+596=1197)~F
wrong outputs(x):  j(849+966=1815)~F
wrong outputs(x):  j(588+811=1399)~F
wrong outputs(x):  j(480+903=1383)~F
wrong outputs(x):  j(525+674=1199)~F
Judgement accuracy of 10000 examples: 9992/10000 (99.92%)
No judging probability of 10000 examples: 0/10000 (0.0%)
True Positive Examples: 4672/10000
False Positive Examples: 0/10000
True Negative Examples: 5320/10000
False Negative Examples: 8/10000
{'carry0': 100.0, 'carry1': 99.88808058198097, 'carry2': 99.91179064980888, 'carry3': 99.925205684368}


In [10]:
tester.test_add_noise()

evaluating addition from: FILE:./test_data/add_noise_judge_prompt.txt


 47%|████▋     | 38/81 [00:00<00:00, 183.75it/s]

wrong outputs(x):  j(616+140=764)~T
wrong outputs(x):  j(195+746=961)~T
wrong outputs(x):  j(407+182=689)~T
wrong outputs(x):  j(912+79=1191)~T
wrong outputs(x):  j(628+263=991)~T
wrong outputs(x):  j(503+349=853)~T
wrong outputs(x):  j(486+196=691)~T
wrong outputs(x):  j(500+402=912)~T
wrong outputs(x):  j(919+16=1035)~T
wrong outputs(x):  j(354+510=874)~T
wrong outputs(x):  j(280+458=938)~T
wrong outputs(x):  j(122+691=823)~T
wrong outputs(x):  j(351+578=930)~T
wrong outputs(x):  j(166+782=958)~T
wrong outputs(x):  j(109+287=486)~T
wrong outputs(x):  j(150+578=828)~T
wrong outputs(x):  j(220+688=909)~T
wrong outputs(x):  j(365+512=977)~T
wrong outputs(x):  j(74+896=1070)~T
wrong outputs(x):  j(548+208=856)~T
wrong outputs(x):  j(64+879=1033)~T
wrong outputs(x):  j(448+123=579)~T
wrong outputs(x):  j(180+419=699)~T
wrong outputs(x):  j(385+188=593)~T
wrong outputs(x):  j(628+164=872)~T
wrong outputs(x):  j(295+191=576)~T
wrong outputs(x):  j(664+298=972)~T
wrong outputs(x):  j(182+697

 94%|█████████▍| 76/81 [00:00<00:00, 182.71it/s]

wrong outputs(x):  j(967+866=1842)~T
wrong outputs(x):  j(714+586=1320)~T
wrong outputs(x):  j(203+972=1205)~T
wrong outputs(x):  j(984+639=1624)~T
wrong outputs(x):  j(349+951=1380)~T
wrong outputs(x):  j(406+195=1101)~T
wrong outputs(x):  j(506+547=1073)~T
wrong outputs(x):  j(866+499=1375)~T
wrong outputs(x):  j(608+690=1498)~T
wrong outputs(x):  j(725+560=1355)~T
wrong outputs(x):  j(725+743=1568)~T
wrong outputs(x):  j(713+575=1488)~T
wrong outputs(x):  j(867+217=1094)~T
wrong outputs(x):  j(538+366=1004)~T
wrong outputs(x):  j(912+720=1642)~T
wrong outputs(x):  j(472+663=1136)~T
wrong outputs(x):  j(900+898=1888)~T
wrong outputs(x):  j(979+198=1186)~T
wrong outputs(x):  j(222+844=1166)~T
wrong outputs(x):  j(718+651=1569)~T
wrong outputs(x):  j(474+991=1555)~T
wrong outputs(x):  j(566+585=1161)~T
wrong outputs(x):  j(784+258=1122)~T
wrong outputs(x):  j(745+696=1521)~T
wrong outputs(x):  j(894+257=1160)~T
wrong outputs(x):  j(603+866=1478)~T
wrong outputs(x):  j(945+786=1740)~T
w

100%|██████████| 81/81 [00:00<00:00, 184.52it/s]

wrong outputs(x):  j(4+411=416)~T
wrong outputs(x):  j(171+1=262)~T
wrong outputs(x):  j(7+306=403)~T
wrong outputs(x):  j(37+93=140)~T
wrong outputs(x):  j(879+8=987)~T
wrong outputs(x):  j(4+100=174)~T
wrong outputs(x):  j(395+3=488)~T
wrong outputs(x):  j(81+11=101)~T
wrong outputs(x):  j(374+7=391)~T
wrong outputs(x):  j(677+7=704)~T
wrong outputs(x):  j(862+9=921)~T
wrong outputs(x):  j(3+889=992)~T
wrong outputs(x):  j(4+271=355)~T
wrong outputs(x):  j(838+6=853)~T
wrong outputs(x):  j(51+39=100)~T
wrong outputs(x):  j(385+4=589)~T
wrong outputs(x):  j(279+5=285)~T
wrong outputs(x):  j(611+8=719)~T
wrong outputs(x):  j(712+8=721)~T
wrong outputs(x):  j(925+8=943)~T
wrong outputs(x):  j(851+2=943)~T
wrong outputs(x):  j(55+13=70)~T
wrong outputs(x):  j(78+17=96)~T
wrong outputs(x):  j(23+60=84)~T
wrong outputs(x):  j(56+15=81)~T
wrong outputs(x):  j(7+17=31)~T
wrong outputs(x):  j(18+8=32)~T
wrong outputs(x):  j(0+34=54)~T
Judgement accuracy of 10000 examples: 9160/10000 (91.60000